In [21]:
# =========================================================
# 0) УСТАНОВКА (COLAB)
# =========================================================
!pip install -q pandas geopandas shapely rasterio tqdm mercantile requests
!pip install -q ultralytics==8.1.0 scipy opencv-python matplotlib

# =========================================================
# 1) ИМПОРТЫ
# =========================================================
import os
import json
import gzip
import glob
import zipfile
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import shutil
from datetime import datetime
from rasterio.warp import transform_bounds
from shapely.geometry import box, shape, Polygon
from shapely.affinity import translate
from shapely.prepared import prep as shp_prep
from shapely.ops import transform as shp_transform
from shapely.validation import make_valid
from tqdm import tqdm
import mercantile
import matplotlib.pyplot as plt
import cv2
import xml.etree.ElementTree as ET

from ultralytics import YOLO
from scipy.optimize import linear_sum_assignment


# =========================================================
# 2) ПУТИ
# =========================================================
# Папка, где лежат .tif / .tiff
INPUT_DIR = "/content/tifs"
# Куда складывать результаты (zip-архивы и временные файлы)
OUT_ROOT  = "/content/out"
os.makedirs(OUT_ROOT, exist_ok=True)

# =========================================================
# 3) НАСТРОЙКИ
# =========================================================
REGION_NAME = "United States"
ZOOM = 9
MIN_AREA_M2 = 5.0
DATASET_LINKS_URL = "https://minedbuildings.z5.web.core.windows.net/global-buildings/dataset-links.csv"

# --- Веса модели ---
BUILDING_MODEL_URL = (
    "https://www.dropbox.com/scl/fi/cdrl62i3mx9p82lqwpik5/"
    "yolov8m-seg_LasVegas.pt?rlkey=8ao7a5zz7xnqfd74deffprix2&dl=1"
)
WEIGHTS_PATH = "/content/models/yolov8m-seg_LasVegas.pt"

# YOLO inference (seg)
YOLO_CONF = 0.50
YOLO_IOU  = 0.60
IMG_TILE = 1024
TILE_OVERLAP = 0.20
GLOBAL_NMS_IOU = 0.50
MAX_DET_PER_TILE = 400
MIN_POLY_AREA_PX = 25
USE_RASTER_COM = False

# Правила матчинга
MAX_DIST_PX = 120.0

# Уточнение сдвига по IoU (маска YOLO против сдвинутого футпринта)
IOU_RADIUS_PX = 18     # 12..30
IOU_ROI_PAD   = 40     # 30..80
DILATE_DET_PX = 1      # 0..2

# CVAT export (лейблы НЕ переводим)
LABEL_BUILDINGS = "Buildings"
LABEL_TODELETE  = "ToDelete"
SIMPLIFY_PX = 1.6
MIN_AREA_PX2 = 20.0

# Сетка для разметчика
GRID_STEP_PX = 500
GRID_LINE_THICKNESS = 1

# Нумерация клеток
# "inside" = писать номер в центре каждой клетки (может слегка мешать объектам)
# "border" = писать координаты по краям (почти не мешает)
GRID_LABEL_MODE = "border"   # "border" | "inside"
GRID_FONT_SCALE = 0.6
GRID_FONT_THICKNESS = 1
GRID_TEXT_MARGIN = 6

# Отчёт
LW_SCALE = 0.5
MAX_DRAW = 6000


# =========================================================
# 4) МЕЛКИЕ ХЕЛПЕРЫ
# =========================================================
def ensure_dirs(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)

def download_weights(url: str, out_path: str, min_bytes_ok=5_000_000):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    if os.path.exists(out_path) and os.path.getsize(out_path) >= min_bytes_ok:
        print("Веса уже скачаны:", out_path, "bytes:", os.path.getsize(out_path))
        return out_path
    print("Скачиваю веса модели...")
    r = requests.get(url, stream=True, timeout=300)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024 * 1024):
            if chunk:
                f.write(chunk)
    size = os.path.getsize(out_path)
    print("Сохранила веса:", out_path, "bytes:", size)
    if size < min_bytes_ok:
        print("⚠️ файл весов выглядит слишком маленьким (возможно, скачалась HTML-страница вместо .pt)")
    return out_path

def get_bbox_wgs84_and_meta(geotiff_path: str):
    with rasterio.open(geotiff_path) as ds:
        bbox_src = ds.bounds
        src_crs = ds.crs
        bbox_wgs84 = transform_bounds(
            src_crs, "EPSG:4326",
            bbox_src.left, bbox_src.bottom, bbox_src.right, bbox_src.top,
            densify_pts=21
        )
        return bbox_wgs84, ds.crs, ds.transform, ds.width, ds.height, ds.profile.copy()

def quadkeys_for_bbox(bbox_wgs84, zoom=9):
    minlon, minlat, maxlon, maxlat = bbox_wgs84
    tiles = list(mercantile.tiles(minlon, minlat, maxlon, maxlat, zooms=[zoom]))
    return sorted({mercantile.quadkey(t) for t in tiles})

def download_file(url: str, out_path: str):
    r = requests.get(url, stream=True, timeout=180)
    r.raise_for_status()
    total = int(r.headers.get("content-length", 0))
    with open(out_path, "wb") as f:
        if total > 0:
            pbar = tqdm(total=total, unit="B", unit_scale=True, desc=os.path.basename(out_path))
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))
            pbar.close()
        else:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
    return out_path

def read_geojsonl_gz_as_gdf(gz_path: str, bbox_wgs84):
    minlon, minlat, maxlon, maxlat = bbox_wgs84
    aoi = box(minlon, minlat, maxlon, maxlat)
    aoi_p = shp_prep(aoi)

    geometries = []
    props_list = []
    with gzip.open(gz_path, "rt", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            feat = json.loads(line)
            geom = shape(feat["geometry"])
            if not geom.is_valid:
                continue
            if not aoi_p.intersects(geom):
                continue
            geometries.append(geom)
            props_list.append(feat.get("properties", {}))

    if len(geometries) == 0:
        return gpd.GeoDataFrame({"geometry": []}, geometry="geometry", crs="EPSG:4326")
    return gpd.GeoDataFrame(props_list, geometry=geometries, crs="EPSG:4326")

def save_geojson_with_crs(gdf: gpd.GeoDataFrame, out_path: str, crs, name="layer"):
    ensure_dirs(out_path)
    geojson_dict = json.loads(gdf.to_json())
    geojson_dict["name"] = name
    try:
        epsg = crs.to_epsg()
        if epsg is not None:
            geojson_dict["crs"] = {"type": "name", "properties": {"name": f"urn:ogc:def:crs:EPSG::{epsg}"}}
    except Exception:
        pass
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(geojson_dict, f, ensure_ascii=False)
    print(f"Сохранила GeoJSON: {out_path} (features={len(gdf)})")

def stretch_uint8(x, p1=2, p2=98):
    x = x.astype(np.float32)
    finite = np.isfinite(x)
    if finite.sum() == 0:
        return np.zeros_like(x, dtype=np.uint8)
    lo, hi = np.percentile(x[finite], [p1, p2])
    if hi <= lo:
        return np.zeros_like(x, dtype=np.uint8)
    y = (x - lo) / (hi - lo)
    return (y * 255).clip(0, 255).astype(np.uint8)

def read_rgb_uint8(geotiff_path: str) -> np.ndarray:
    with rasterio.open(geotiff_path) as ds:
        if ds.count >= 3:
            r = ds.read(1); g = ds.read(2); b = ds.read(3)
        else:
            band = ds.read(1); r = g = b = band
    return np.stack([stretch_uint8(r), stretch_uint8(g), stretch_uint8(b)], axis=-1)

def save_png_rgb(path: str, img_rgb_uint8: np.ndarray):
    ensure_dirs(path)
    bgr = cv2.cvtColor(img_rgb_uint8, cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, bgr)

def draw_grid_numbered(img_rgb_uint8: np.ndarray,
                       step=500,
                       thickness=1,
                       font_scale=0.8,
                       font_thickness=2,
                       margin=8) -> np.ndarray:
    """
    Рисует белую сетку и НУМЕРАЦИЮ клеток 1,2,3,... в левом верхнем углу каждой клетки.
    Порядок нумерации: слева-направо, сверху-вниз.
    """
    out = img_rgb_uint8.copy()
    H, W = out.shape[:2]

    # 1) Сетка (белая)
    for x in range(0, W, step):
        cv2.line(out, (x, 0), (x, H - 1), (255, 255, 255), thickness=thickness)
    for y in range(0, H, step):
        cv2.line(out, (0, y), (W - 1, y), (255, 255, 255), thickness=thickness)

    # 2) Нумерация клеток
    font = cv2.FONT_HERSHEY_SIMPLEX

    def put_text(x, y, text):
        # белая "обводка", чтобы читалось на любом фоне
        cv2.putText(out, text, (x, y), font, font_scale, (255, 255, 255), font_thickness + 3, cv2.LINE_AA)
        # чёрный текст
        cv2.putText(out, text, (x, y), font, font_scale, (0, 0, 0), font_thickness, cv2.LINE_AA)

    ncols = int(np.ceil(W / step))
    nrows = int(np.ceil(H / step))

    k = 1
    for r in range(nrows):
        for c in range(ncols):
            x0 = c * step
            y0 = r * step
            # Точка текста — чуть правее/ниже угла клетки (y — это baseline)
            tx = int(min(x0 + margin, W - 1))
            ty = int(min(y0 + margin + 18, H - 1))
            put_text(tx, ty, str(k))
            k += 1

    return out

def tile_grid(H, W, tile, overlap):
    stride = int(tile * (1.0 - overlap))
    stride = max(1, stride)
    for y0 in range(0, H, stride):
        for x0 in range(0, W, stride):
            y1 = min(H, y0 + tile)
            x1 = min(W, x0 + tile)
            if (y1 - y0) < 64 or (x1 - x0) < 64:
                continue
            yield x0, y0, x1, y1

def box_iou_xyxy(a, b):
    xA = max(a[0], b[0]); yA = max(a[1], b[1])
    xB = min(a[2], b[2]); yB = min(a[3], b[3])
    interW = max(0.0, xB - xA); interH = max(0.0, yB - yA)
    inter = interW * interH
    if inter <= 0:
        return 0.0
    areaA = max(0.0, a[2]-a[0]) * max(0.0, a[3]-a[1])
    areaB = max(0.0, b[2]-b[0]) * max(0.0, b[3]-b[1])
    union = areaA + areaB - inter
    return float(inter / union) if union > 0 else 0.0

def nms_xyxy(boxes, scores, iou_thr=0.5):
    if len(boxes) == 0:
        return []
    idxs = np.argsort(scores)[::-1]
    keep = []
    while len(idxs) > 0:
        i = idxs[0]
        keep.append(i)
        rest = idxs[1:]
        new_rest = []
        for j in rest:
            if box_iou_xyxy(boxes[i], boxes[j]) < iou_thr:
                new_rest.append(j)
        idxs = np.array(new_rest, dtype=int)
    return keep

def zip_files(zip_path: str, files: list, arcname_prefix: str = ""):
    ensure_dirs(zip_path)
    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
        for f in files:
            if not f or (not os.path.exists(f)):
                continue
            arc = os.path.join(arcname_prefix, os.path.basename(f)) if arcname_prefix else os.path.basename(f)
            z.write(f, arcname=arc)
    print("Сохранила zip:", zip_path)

def base_name_no_ext(path: str) -> str:
    b = os.path.basename(path)
    for ext in (".tif", ".tiff", ".TIF", ".TIFF"):
        if b.endswith(ext):
            return b[: -len(ext)]
    return os.path.splitext(b)[0]


# =========================================================
# 5) YOLO SEGMENTATION (TILED) + CENTERS
# =========================================================
def _poly_centroid_xy(poly_xy: np.ndarray):
    if poly_xy is None or len(poly_xy) < 3:
        return None
    try:
        poly = Polygon(poly_xy)
        if not poly.is_valid:
            poly = make_valid(poly)
            if poly.geom_type == "MultiPolygon":
                poly = max(list(poly.geoms), key=lambda g: g.area)
        if poly.is_empty or poly.area <= 1e-6:
            return None
        c = poly.centroid
        return float(c.x), float(c.y)
    except Exception:
        return None

def _poly_center_of_mass_raster(poly_xy: np.ndarray, tile_h: int, tile_w: int):
    if poly_xy is None or len(poly_xy) < 3:
        return None
    mask = np.zeros((tile_h, tile_w), dtype=np.uint8)
    pts = np.round(poly_xy).astype(np.int32)
    pts[:,0] = np.clip(pts[:,0], 0, tile_w-1)
    pts[:,1] = np.clip(pts[:,1], 0, tile_h-1)
    cv2.fillPoly(mask, [pts], 1)
    ys, xs = np.nonzero(mask)
    if len(xs) == 0:
        return None
    return float(xs.mean()), float(ys.mean())

def yolo_seg_tiled(
    img_rgb_uint8,
    model,
    conf,
    iou,
    tile=1024,
    overlap=0.2,
    global_nms_iou=0.5,
    max_det_tile=400,
    min_poly_area_px=25,
    use_raster_com=False,
):
    H, W = img_rgb_uint8.shape[:2]
    dets_all = []

    for x0, y0, x1, y1 in tile_grid(H, W, tile, overlap):
        tile_img = img_rgb_uint8[y0:y1, x0:x1]
        th, tw = tile_img.shape[:2]

        res = model.predict(tile_img, conf=conf, iou=iou, verbose=False)[0]
        if res.boxes is None or len(res.boxes) == 0 or res.masks is None:
            continue

        confs = res.boxes.conf.cpu().numpy().astype(np.float32)
        polys = res.masks.xy
        if polys is None or len(polys) == 0:
            continue

        if len(confs) > max_det_tile:
            order = np.argsort(confs)[::-1][:max_det_tile]
        else:
            order = np.arange(len(confs))

        for k in order:
            poly = polys[k]
            if poly is None or len(poly) < 3:
                continue
            poly = np.asarray(poly, dtype=np.float32)

            # фильтр по площади
            try:
                shp = Polygon(poly)
                if not shp.is_valid:
                    shp = make_valid(shp)
                    if shp.geom_type == "MultiPolygon":
                        shp = max(list(shp.geoms), key=lambda g: g.area)
                area = float(shp.area) if (not shp.is_empty) else 0.0
            except Exception:
                area = 0.0
            if area < min_poly_area_px:
                continue

            # центр
            if use_raster_com:
                cent = _poly_center_of_mass_raster(poly, th, tw)
            else:
                cent = _poly_centroid_xy(poly)
            if cent is None:
                continue
            cx_t, cy_t = cent

            xs = poly[:,0]; ys = poly[:,1]
            bx1, by1, bx2, by2 = float(xs.min()), float(ys.min()), float(xs.max()), float(ys.max())
            if (bx2 - bx1) <= 1 or (by2 - by1) <= 1:
                continue

            poly_full = poly.copy()
            poly_full[:,0] += x0
            poly_full[:,1] += y0

            x1f = float(np.clip(bx1 + x0, 0, W-1))
            y1f = float(np.clip(by1 + y0, 0, H-1))
            x2f = float(np.clip(bx2 + x0, 0, W-1))
            y2f = float(np.clip(by2 + y0, 0, H-1))
            if x2f <= x1f or y2f <= y1f:
                continue

            dets_all.append({
                "x1": x1f, "y1": y1f, "x2": x2f, "y2": y2f,
                "cx": float(cx_t + x0), "cy": float(cy_t + y0),
                "conf": float(confs[k]),
                "poly": poly_full
            })

    if len(dets_all) == 0:
        return []

    boxes = np.array([[d["x1"], d["y1"], d["x2"], d["y2"]] for d in dets_all], dtype=np.float32)
    scores = np.array([d["conf"] for d in dets_all], dtype=np.float32)
    keep = nms_xyxy(boxes, scores, iou_thr=global_nms_iou)
    return [dets_all[i] for i in keep]


# =========================================================
# 6) FOOTPRINTS -> PX RECORDS + MATCHING
# =========================================================
def footprints_to_px_records(gdf, geotiff_path):
    with rasterio.open(geotiff_path) as ds:
        inv_aff = ~ds.transform
        W, H = ds.width, ds.height

    recs = []
    for idx, geom in enumerate(gdf.geometry):
        if geom is None or geom.is_empty or (not geom.is_valid):
            continue
        minx, miny, maxx, maxy = geom.bounds
        x1, y1 = inv_aff * (minx, miny)
        x2, y2 = inv_aff * (maxx, maxy)

        x_min, x_max = (x1, x2) if x1 <= x2 else (x2, x1)
        y_min, y_max = (y1, y2) if y1 <= y2 else (y2, y1)

        x_min = float(np.clip(x_min, 0, W-1)); x_max = float(np.clip(x_max, 0, W-1))
        y_min = float(np.clip(y_min, 0, H-1)); y_max = float(np.clip(y_max, 0, H-1))
        if x_max <= x_min or y_max <= y_min:
            continue

        recs.append({
            "fp_idx": idx,
            "x1": x_min, "y1": y_min, "x2": x_max, "y2": y_max,
            "cx": 0.5*(x_min+x_max), "cy": 0.5*(y_min+y_max)
        })
    return recs

def bbox_intersects(a, b):
    xA = max(a[0], b[0]); yA = max(a[1], b[1])
    xB = min(a[2], b[2]); yB = min(a[3], b[3])
    return (xB - xA) > 0 and (yB - yA) > 0

def match_overlap_then_min_center(dets, fps, max_dist_px=None, huge_cost=1e9):
    if len(dets) == 0 or len(fps) == 0:
        return [], set([f["fp_idx"] for f in fps]), set(range(len(dets)))

    D = np.full((len(fps), len(dets)), fill_value=huge_cost, dtype=np.float32)

    for i, f in enumerate(fps):
        fb = [f["x1"], f["y1"], f["x2"], f["y2"]]
        for j, d in enumerate(dets):
            db = [d["x1"], d["y1"], d["x2"], d["y2"]]
            if bbox_intersects(fb, db):
                dist = float(np.hypot(f["cx"] - d["cx"], f["cy"] - d["cy"]))
                if (max_dist_px is None) or (dist <= max_dist_px):
                    D[i, j] = dist

    row_ind, col_ind = linear_sum_assignment(D)

    matches = []
    matched_fp = set()
    matched_det = set()
    for r, c in zip(row_ind, col_ind):
        dist = float(D[r, c])
        if dist >= huge_cost * 0.5:
            continue
        fp_idx = fps[r]["fp_idx"]
        matches.append((fp_idx, c, dist))
        matched_fp.add(fp_idx)
        matched_det.add(c)

    all_fp = set([f["fp_idx"] for f in fps])
    all_det = set(range(len(dets)))
    return matches, (all_fp - matched_fp), (all_det - matched_det)

def pixel_delta_to_world_delta(transform, dx_px, dy_px):
    x0, y0 = transform * (0, 0)
    x1, y1 = transform * (dx_px, dy_px)
    return (x1 - x0), (y1 - y0)


# =========================================================
# 7) УТОЧНЕНИЕ СДВИГА ПО IoU (против YOLO масок)
# =========================================================
def rasterize_poly(poly_xy, h, w):
    mask = np.zeros((h, w), dtype=np.uint8)
    if poly_xy is None or len(poly_xy) < 3:
        return mask
    pts = np.round(np.asarray(poly_xy)).astype(np.int32)
    pts[:, 0] = np.clip(pts[:, 0], 0, w - 1)
    pts[:, 1] = np.clip(pts[:, 1], 0, h - 1)
    cv2.fillPoly(mask, [pts], 1)
    return mask

def iou_masks(a, b):
    inter = np.logical_and(a, b).sum()
    if inter == 0:
        return 0.0
    union = np.logical_or(a, b).sum()
    return float(inter / union) if union > 0 else 0.0

def geom_to_px_poly(geom, inv_aff):
    if geom is None or geom.is_empty:
        return None
    if geom.geom_type == "Polygon":
        return np.array([inv_aff * (x, y) for x, y in geom.exterior.coords], dtype=np.float32)
    if geom.geom_type == "MultiPolygon":
        poly = max(list(geom.geoms), key=lambda g: g.area)
        return np.array([inv_aff * (x, y) for x, y in poly.exterior.coords], dtype=np.float32)
    return None

def crop_roi_from_bbox(x1, y1, x2, y2, W, H, pad=40):
    x1 = int(max(0, np.floor(x1) - pad))
    y1 = int(max(0, np.floor(y1) - pad))
    x2 = int(min(W, np.ceil(x2) + pad))
    y2 = int(min(H, np.ceil(y2) + pad))
    if x2 <= x1 + 8 or y2 <= y1 + 8:
        return None
    return x1, y1, x2, y2

def best_shift_by_iou(fp_poly_px, det_poly_px, W, H, dx0, dy0, radius=18, pad=40, dilate_det=1):
    if fp_poly_px is None or det_poly_px is None or len(fp_poly_px) < 3 or len(det_poly_px) < 3:
        return dx0, dy0, 0.0

    allx = np.concatenate([fp_poly_px[:, 0], det_poly_px[:, 0]])
    ally = np.concatenate([fp_poly_px[:, 1], det_poly_px[:, 1]])
    roi = crop_roi_from_bbox(allx.min(), ally.min(), allx.max(), ally.max(), W, H, pad=pad)
    if roi is None:
        return dx0, dy0, 0.0

    x0, y0, x1, y1 = roi
    rh, rw = (y1 - y0), (x1 - x0)

    det_local = det_poly_px.copy()
    det_local[:, 0] -= x0
    det_local[:, 1] -= y0
    det_mask = rasterize_poly(det_local, rh, rw)

    if dilate_det and dilate_det > 0:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * dilate_det + 1, 2 * dilate_det + 1))
        det_mask = cv2.dilate(det_mask, k, iterations=1)

    dx0i = int(np.round(dx0))
    dy0i = int(np.round(dy0))
    r = int(max(0, radius))

    best_iou = -1.0
    best_dx, best_dy = dx0i, dy0i

    for dy in range(dy0i - r, dy0i + r + 1):
        for dx in range(dx0i - r, dx0i + r + 1):
            fp_local = fp_poly_px.copy()
            fp_local[:, 0] = fp_local[:, 0] + dx - x0
            fp_local[:, 1] = fp_local[:, 1] + dy - y0
            fp_mask = rasterize_poly(fp_local, rh, rw)
            sc = iou_masks(fp_mask, det_mask)
            if sc > best_iou:
                best_iou = sc
                best_dx, best_dy = dx, dy

    return float(best_dx), float(best_dy), float(best_iou)

def apply_shifts_by_iou(gdf, geotiff_path, fps_records, dets, matches,
                        radius=18, pad=40, dilate_det=1):
    with rasterio.open(geotiff_path) as ds:
        transform = ds.transform
        inv_aff = ~ds.transform
        W, H = ds.width, ds.height

    fp_by_idx = {r["fp_idx"]: r for r in fps_records}
    gdf_shifted = gdf.copy()
    rows = []

    for fp_idx, det_j, center_dist in matches:
        f = fp_by_idx[fp_idx]
        d = dets[det_j]

        dx0 = float(d["cx"] - f["cx"])
        dy0 = float(d["cy"] - f["cy"])

        geom = gdf.geometry.iloc[fp_idx]
        fp_poly_px = geom_to_px_poly(geom, inv_aff)
        det_poly_px = d.get("poly", None)

        dx_px, dy_px, best_iou = best_shift_by_iou(
            fp_poly_px=fp_poly_px,
            det_poly_px=det_poly_px,
            W=W, H=H,
            dx0=dx0, dy0=dy0,
            radius=radius,
            pad=pad,
            dilate_det=dilate_det
        )

        dx_w, dy_w = pixel_delta_to_world_delta(transform, dx_px, dy_px)
        gdf_shifted.at[fp_idx, "geometry"] = translate(geom, xoff=dx_w, yoff=dy_w)

        rows.append({
            "fp_idx": fp_idx,
            "det_j": int(det_j),
            "center_dist_px": float(center_dist),
            "dx0_px": dx0, "dy0_px": dy0,
            "dx_px": dx_px, "dy_px": dy_px,
            "best_iou": float(best_iou),
            "dx_world": dx_w, "dy_world": dy_w,
            "det_conf": d.get("conf", None)
        })

    return gdf_shifted, pd.DataFrame(rows)


# =========================================================
# 8) ЭКСПОРТ CVAT (2 лейбла: Buildings + ToDelete)
# =========================================================
def gdf_to_pixel_polygons(gdf, geotiff_path, simplify_px=1.6, min_area_px2=20.0):
    with rasterio.open(geotiff_path) as ds:
        W, H = ds.width, ds.height
        affine = ds.transform
        tif_crs = ds.crs

        def to_pixel(x, y, z=None):
            col, row = ~affine * (x, y)
            return (col, row)

    g = gdf.copy()
    if g.crs is None:
        g = g.set_crs(tif_crs, allow_override=True)
    elif g.crs != tif_crs:
        g = g.to_crs(tif_crs)

    polys_out = []
    for geom in g.geometry:
        if geom is None or geom.is_empty:
            continue

        parts = []
        if geom.geom_type == "Polygon":
            parts = [geom]
        elif geom.geom_type == "MultiPolygon":
            parts = list(geom.geoms)
        else:
            continue

        for poly in parts:
            poly_px = shp_transform(to_pixel, poly)
            poly_px = poly_px.simplify(simplify_px, preserve_topology=True)
            if poly_px.is_empty or poly_px.area < min_area_px2:
                continue

            coords = list(poly_px.exterior.coords)
            if len(coords) > 2 and coords[0] == coords[-1]:
                coords = coords[:-1]
            if len(coords) < 3:
                continue

            clamped = []
            for x, y in coords:
                x = 0.0 if x < 0 else (W - 1.0 if x > W - 1 else x)
                y = 0.0 if y < 0 else (H - 1.0 if y > H - 1 else y)
                clamped.append((float(x), float(y)))

            polys_out.append(clamped)

    return polys_out

def export_cvat_two_labels(
    geotiff_path: str,
    gdf_buildings: gpd.GeoDataFrame,
    gdf_todelete: gpd.GeoDataFrame,
    out_xml_path: str,
    label_buildings: str = "Buildings",
    label_todelete: str = "ToDelete",
    simplify_px: float = 1.6,
    min_area_px2: float = 20.0,
):
    with rasterio.open(geotiff_path) as ds:
        W, H = ds.width, ds.height

    polys_build = gdf_to_pixel_polygons(gdf_buildings, geotiff_path, simplify_px=simplify_px, min_area_px2=min_area_px2)
    polys_del   = gdf_to_pixel_polygons(gdf_todelete, geotiff_path, simplify_px=simplify_px, min_area_px2=min_area_px2)

    ann = ET.Element("annotations")
    meta = ET.SubElement(ann, "meta")
    task = ET.SubElement(meta, "task")
    labels = ET.SubElement(task, "labels")

    lab1 = ET.SubElement(labels, "label")
    ET.SubElement(lab1, "name").text = label_buildings

    lab2 = ET.SubElement(labels, "label")
    ET.SubElement(lab2, "name").text = label_todelete

    img = ET.SubElement(ann, "image", {
        "id": "0",
        "name": os.path.basename(geotiff_path),
        "width": str(W),
        "height": str(H)
    })

    obj_id = 0
    def add_polys(polys, label_name):
        nonlocal obj_id
        for pts in polys:
            points_str = ";".join([f"{x:.2f},{y:.2f}" for x, y in pts])
            ET.SubElement(img, "polygon", {
                "label": label_name,
                "occluded": "0",
                "source": "auto",
                "z_order": "0",
                "points": points_str,
                "group_id": str(obj_id),
            })
            obj_id += 1

    add_polys(polys_build, label_buildings)
    add_polys(polys_del, label_todelete)

    ensure_dirs(out_xml_path)
    tree = ET.ElementTree(ann)
    tree.write(out_xml_path, encoding="utf-8", xml_declaration=True)
    print(f"Сохранила CVAT XML: {out_xml_path} | Buildings: {len(polys_build)} | ToDelete: {len(polys_del)} | всего: {obj_id}")


# =========================================================
# 9) PNG-ОТЧЁТ (зелёный/красный) + русская легенда
# =========================================================
def polygon_to_px_lines(geom, inv_aff):
    out = []
    if geom is None or geom.is_empty:
        return out
    if geom.geom_type == "Polygon":
        coords = list(geom.exterior.coords)
        xs, ys = [], []
        for x, y in coords:
            col, row = inv_aff * (x, y)
            xs.append(col); ys.append(row)
        out.append((xs, ys))
    elif geom.geom_type == "MultiPolygon":
        for p in geom.geoms:
            out.extend(polygon_to_px_lines(p, inv_aff))
    return out

def draw_report_buildings_todelete(img, geotiff_path, gdf_buildings, gdf_todelete, out_png, max_draw=6000, lw_scale=0.5):
    with rasterio.open(geotiff_path) as ds:
        inv_aff = ~ds.transform

    plt.figure(figsize=(14, 14))
    ax = plt.gca()
    ax.imshow(img)
    ax.axis("off")

    n = 0
    for geom in gdf_buildings.geometry:
        if n >= max_draw:
            break
        for xs, ys in polygon_to_px_lines(geom, inv_aff):
            ax.plot(xs, ys, linewidth=0.9 * lw_scale, color="lime", alpha=0.90)
        n += 1

    n = 0
    for geom in gdf_todelete.geometry:
        if n >= max_draw:
            break
        for xs, ys in polygon_to_px_lines(geom, inv_aff):
            ax.plot(xs, ys, linewidth=1.2 * lw_scale, color="red", alpha=0.95)
        n += 1

    from matplotlib.lines import Line2D
    legend = [
        Line2D([0],[0], color="lime", lw=2*lw_scale, label="Buildings"),
        Line2D([0],[0], color="red",  lw=2*lw_scale, label="ToDelete"),
    ]
    ax.legend(handles=legend, loc="lower left", framealpha=0.85)

    ensure_dirs(out_png)
    plt.tight_layout()
    plt.savefig(out_png, dpi=200)
    plt.close()
    print("Сохранила PNG-отчёт:", out_png)


# =========================================================
# 10) ЗАГРУЗКА MS BUILDINGS (RAW FOOTPRINTS)
# =========================================================
def load_ms_buildings_as_gdf(geotiff_path: str, work_dir: str):
    bbox_wgs84, tif_crs, tif_transform, w, h, profile = get_bbox_wgs84_and_meta(geotiff_path)
    print("GeoTIFF:", geotiff_path)
    print("Размер:", w, "x", h)
    print("CRS:", tif_crs)
    print("AOI bbox WGS84:", bbox_wgs84)

    qks = quadkeys_for_bbox(bbox_wgs84, zoom=ZOOM)
    qks_norm = [q.zfill(ZOOM) for q in qks]
    print(f"Quadkeys (zoom={ZOOM}) для AOI:", qks_norm)

    print("Загружаю dataset-links.csv ...")
    df = pd.read_csv(DATASET_LINKS_URL, dtype=str)
    cols = {c.lower(): c for c in df.columns}
    loc_col = cols.get("location")
    qk_col  = cols.get("quadkey")
    url_col = cols.get("url")
    if loc_col is None or qk_col is None or url_col is None:
        raise RuntimeError(f"Неожиданные колонки в dataset-links.csv: {list(df.columns)}")

    df[qk_col] = df[qk_col].astype(str).str.strip().str.zfill(ZOOM)
    df[loc_col] = df[loc_col].astype(str).str.strip()

    sub = df[(df[loc_col] == REGION_NAME) & (df[qk_col].isin(qks_norm))].copy()
    print(f"Найдено строк манифеста: {len(sub)}")

    if len(sub) == 0:
        sub2 = df[df[qk_col].isin(qks_norm)].copy()
        print(f"Fallback по одному QuadKey: {len(sub2)}")
        if len(sub2) > 0:
            print("Локации для этих quadkey:", sub2[loc_col].unique()[:10])
            sub = sub2

    if len(sub) == 0:
        print("⚠️ Не нашла подходящих ссылок. Попробуй поменять REGION_NAME или ZOOM.")
        return gpd.GeoDataFrame({"class": [], "geometry": []}, geometry="geometry", crs=tif_crs), tif_crs

    urls = sub[url_col].tolist()
    print("Файлов к скачиванию:", len(urls))

    all_gdfs = []
    for i, url in enumerate(urls, 1):
        local_gz = f"{work_dir}/ms_buildings_{i}.geojsonl.gz"
        if not os.path.exists(local_gz):
            print(f"\n[{i}/{len(urls)}] Скачивание...")
            download_file(url, local_gz)
        else:
            print(f"\n[{i}/{len(urls)}] Кэш:", local_gz)

        print(f"[{i}/{len(urls)}] Парсинг GeoJSONL ...")
        gdf_part = read_geojsonl_gz_as_gdf(local_gz, bbox_wgs84)
        print(f"    -> оставлено объектов: {len(gdf_part)}")
        if len(gdf_part) > 0:
            all_gdfs.append(gdf_part)

    if len(all_gdfs) == 0:
        print("⚠️ После клипа не осталось футпринтов.")
        return gpd.GeoDataFrame({"class": [], "geometry": []}, geometry="geometry", crs=tif_crs), tif_crs

    gdf_wgs = gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index=True), geometry="geometry", crs="EPSG:4326")
    print("Всего футпринтов (WGS84):", len(gdf_wgs))

    gdf = gdf_wgs.to_crs(tif_crs)
    gdf = gdf[gdf.geometry.notnull()]
    gdf = gdf[gdf.is_valid].reset_index(drop=True)

    if MIN_AREA_M2 and MIN_AREA_M2 > 0:
        gdf = gdf[gdf.geometry.area >= MIN_AREA_M2]

    gdf = gdf.reset_index(drop=True)
    print("Всего футпринтов после фильтра по площади:", len(gdf))

    gdf["class"] = 1
    return gdf[["class", "geometry"]], tif_crs


# =========================================================
# 11) НАДЁЖНАЯ ЗАГРУЗКА МОДЕЛИ (PyTorch 2.6+)
# =========================================================
def load_ultralytics_model_robust(weights_path: str):
    import torch
    import torch.nn as nn

    def patch_safe_globals():
        safe = []
        safe += [
            nn.Sequential, nn.ModuleList, nn.ModuleDict,
            nn.Conv2d, nn.BatchNorm2d, nn.GroupNorm, nn.LayerNorm,
            nn.ReLU, nn.ReLU6, nn.SiLU, nn.LeakyReLU, nn.Hardswish,
            nn.Upsample, nn.MaxPool2d, nn.AvgPool2d, nn.AdaptiveAvgPool2d,
            nn.Dropout, nn.Identity,
        ]
        try:
            from ultralytics.nn.tasks import SegmentationModel, DetectionModel, ClassificationModel, PoseModel, OBBModel
            safe += [SegmentationModel, DetectionModel, ClassificationModel, PoseModel, OBBModel]
        except Exception:
            pass
        try:
            from ultralytics.nn.modules.conv import Conv, DWConv, ConvTranspose, GhostConv, RepConv
            safe += [Conv, DWConv, ConvTranspose, GhostConv, RepConv]
        except Exception:
            pass
        try:
            from ultralytics.nn.modules.block import (
                C2f, C3, C3TR, C3Ghost, Bottleneck, BottleneckCSP,
                SPP, SPPF, Focus, GhostBottleneck, HGBlock, HGStem
            )
            safe += [C2f, C3, C3TR, C3Ghost, Bottleneck, BottleneckCSP, SPP, SPPF,
                     Focus, GhostBottleneck, HGBlock, HGStem]
        except Exception:
            pass
        try:
            from ultralytics.nn.modules.head import Detect, Segment, Pose, Classify, OBB
            safe += [Detect, Segment, Pose, Classify, OBB]
        except Exception:
            pass
        try:
            from ultralytics.nn.modules.transformer import TransformerBlock
            safe += [TransformerBlock]
        except Exception:
            pass

        torch.serialization.add_safe_globals(safe)
        print(f"✅ Добавила safe globals: {len(safe)}")

    try:
        patch_safe_globals()
        m = YOLO(weights_path)
        print("✅ Модель загружена (safe-globals)")
        return m
    except Exception as e1:
        print("⚠️ Safe-globals не сработало -> fallback weights_only=False.", repr(e1))

    import torch
    _old_load = torch.load

    def _torch_load_unsafe(*args, **kwargs):
        kwargs["weights_only"] = False
        return _old_load(*args, **kwargs)

    torch.load = _torch_load_unsafe
    try:
        m = YOLO(weights_path)
        print("✅ Модель загружена (weights_only=False fallback)")
        return m
    finally:
        torch.load = _old_load


# =========================================================
# 12) ОБРАБОТКА ОДНОГО TIFF
# =========================================================
def run_pipeline_for_file(geotiff_path: str, model):
    base = base_name_no_ext(geotiff_path)
    work_dir = os.path.join(OUT_ROOT, base)
    os.makedirs(work_dir, exist_ok=True)

    # файлы для служебного архива
    out_geojson_raw      = f"{work_dir}/{base}_raw.geojson"
    out_geojson_shifted  = f"{work_dir}/{base}_Buildings_shifted.geojson"
    out_geojson_todelete = f"{work_dir}/{base}_ToDelete.geojson"
    out_shifts_csv       = f"{work_dir}/{base}_shifts_report.csv"

    # файлы для основного архива (разметчик)
    out_cvat_xml         = f"{work_dir}/{base}_cvat_2labels.xml"
    out_grid_png         = f"{work_dir}/{base}_grid.png"
    out_report_png       = f"{work_dir}/{base}_report_buildings_todelete.png"

    # A) футпринты
    gdf_raw, tif_crs = load_ms_buildings_as_gdf(geotiff_path, work_dir=work_dir)
    save_geojson_with_crs(gdf_raw, out_geojson_raw, tif_crs, name="footprints_raw")

    # B) картинка + сетка + нумерация
    img = read_rgb_uint8(geotiff_path)
    img_grid = draw_grid_numbered(
        img_rgb_uint8=img,
        step=GRID_STEP_PX,
        thickness=GRID_LINE_THICKNESS,
        font_scale=GRID_FONT_SCALE,
        font_thickness=GRID_FONT_THICKNESS,
        margin=GRID_TEXT_MARGIN
    )

    save_png_rgb(out_grid_png, img_grid)

    # C) YOLO сегментация
    print("Запускаю YOLO segmentation (тайлы)...")
    dets = yolo_seg_tiled(
        img_rgb_uint8=img,
        model=model,
        conf=YOLO_CONF,
        iou=YOLO_IOU,
        tile=IMG_TILE,
        overlap=TILE_OVERLAP,
        global_nms_iou=GLOBAL_NMS_IOU,
        max_det_tile=MAX_DET_PER_TILE,
        min_poly_area_px=MIN_POLY_AREA_PX,
        use_raster_com=USE_RASTER_COM
    )
    print("Детекций после склейки+NMS:", len(dets))

    # D) футпринты -> пиксельные записи
    fps = footprints_to_px_records(gdf_raw, geotiff_path)
    print("Футпринтов для матчинга:", len(fps))

    # E) матчинг футпринтов и детекций
    matches, unmatched_fp, unmatched_det = match_overlap_then_min_center(
        dets=dets, fps=fps, max_dist_px=MAX_DIST_PX, huge_cost=1e9
    )
    print("Матчей:", len(matches))
    print("Футпринтов без пары (ToDelete):", len(unmatched_fp))
    print("Детекций без пары (игнорируем):", len(unmatched_det))

    # F) сдвиг матченных футпринтов (Buildings)
    print("Уточняю сдвиги по IoU против YOLO масок...")
    gdf_shifted, shifts_df = apply_shifts_by_iou(
        gdf=gdf_raw,
        geotiff_path=geotiff_path,
        fps_records=fps,
        dets=dets,
        matches=matches,
        radius=IOU_RADIUS_PX,
        pad=IOU_ROI_PAD,
        dilate_det=DILATE_DET_PX
    )
    shifts_df.to_csv(out_shifts_csv, index=False)
    print("Сохранила отчёт по сдвигам:", out_shifts_csv)

    # G) делим на Buildings vs ToDelete
    todelete_idxs = sorted(list(unmatched_fp))
    keep_mask = np.ones(len(gdf_shifted), dtype=bool)
    for idx in todelete_idxs:
        if 0 <= idx < len(keep_mask):
            keep_mask[idx] = False

    gdf_buildings = gdf_shifted.loc[keep_mask].reset_index(drop=True)
    gdf_todelete  = gdf_raw.iloc[todelete_idxs].copy().reset_index(drop=True)

    save_geojson_with_crs(gdf_buildings, out_geojson_shifted, tif_crs, name="Buildings_shifted")
    save_geojson_with_crs(gdf_todelete,  out_geojson_todelete, tif_crs, name="ToDelete_unmatched")

    # H) CVAT XML (2 лейбла)
    export_cvat_two_labels(
        geotiff_path=geotiff_path,
        gdf_buildings=gdf_buildings,
        gdf_todelete=gdf_todelete,
        out_xml_path=out_cvat_xml,
        label_buildings=LABEL_BUILDINGS,
        label_todelete=LABEL_TODELETE,
        simplify_px=SIMPLIFY_PX,
        min_area_px2=MIN_AREA_PX2
    )

    # I) PNG-отчёт (зелёный/красный) — чтобы разметчик сразу оценил качество
    draw_report_buildings_todelete(
        img=img,
        geotiff_path=geotiff_path,
        gdf_buildings=gdf_buildings,
        gdf_todelete=gdf_todelete,
        out_png=out_report_png,
        max_draw=MAX_DRAW,
        lw_scale=LW_SCALE
    )

    # J) Архивы
    # Основной: <имя_снимка>.zip -> сетка + CVAT + PNG-отчёт
    main_zip = os.path.join(OUT_ROOT, f"{base}.zip")
    zip_files(
        zip_path=main_zip,
        files=[out_grid_png, out_cvat_xml, out_report_png],
        arcname_prefix=base
    )

    # Extras: <имя_снимка>_extras.zip -> geojson + csv
    extras_zip = os.path.join(OUT_ROOT, f"{base}_extras.zip")
    zip_files(
        zip_path=extras_zip,
        files=[out_geojson_raw, out_geojson_shifted, out_geojson_todelete, out_shifts_csv],
        arcname_prefix=f"{base}_extras"
    )

    # K) Удаляем рабочую папку с незапакованными файлами
    try:
        shutil.rmtree(work_dir)
        print("Удалена временная папка:", work_dir)
    except Exception as e:
        print("⚠️ Не смогла удалить временную папку:", work_dir, "Причина:", repr(e))

    return {
        "base": base,
        "work_dir": work_dir,
        "main_zip": main_zip,
        "extras_zip": extras_zip,
        "grid_png": out_grid_png,
        "cvat_xml": out_cvat_xml,
        "report_png": out_report_png,

        # метрики для общего отчёта
        "n_footprints_raw": int(len(gdf_raw)),
        "n_buildings": int(len(gdf_buildings)),
        "n_todelete": int(len(gdf_todelete)),
        "n_detections": int(len(dets)),
        "n_matches": int(len(matches)),
        "n_unmatched_fp": int(len(unmatched_fp)),
        "n_unmatched_det": int(len(unmatched_det)),
    }


# =========================================================
# 13) ЗАПУСК ПО ВСЕМ TIFF В ПАПКЕ
# =========================================================
def run_all():
    tifs = sorted(glob.glob(os.path.join(INPUT_DIR, "*.tif"))) + sorted(glob.glob(os.path.join(INPUT_DIR, "*.tiff")))
    if len(tifs) == 0:
        raise FileNotFoundError(f"В папке нет .tif/.tiff: {INPUT_DIR}")

    # Один раз: веса + модель
    download_weights(BUILDING_MODEL_URL, WEIGHTS_PATH)
    model = load_ultralytics_model_robust(WEIGHTS_PATH)

    results = []
    for geotiff_path in tifs:
        print("\n" + "="*90)
        print("ОБРАБАТЫВАЮ:", geotiff_path)
        print("="*90)
        try:
            res = run_pipeline_for_file(geotiff_path, model=model)
            results.append(res)
            print("✅ Готово:", res["base"])
        except Exception as e:
            print("❌ Ошибка на файле:", geotiff_path)
            print("Причина:", repr(e))

    # Общий XLSX-репорт по всем снимкам
    if len(results) > 0:
        df_rep = pd.DataFrame(results)

        # Немного удобных вычислений
        if "n_buildings" in df_rep.columns and "n_todelete" in df_rep.columns:
            df_rep["n_total_polygons_for_cvat"] = df_rep["n_buildings"] + df_rep["n_todelete"]

        # Сортировка по имени
        if "base" in df_rep.columns:
            df_rep = df_rep.sort_values("base").reset_index(drop=True)

        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        out_xlsx = os.path.join(OUT_ROOT, f"summary_buildings_{ts}.xlsx")

        # Пишем Excel: лист "summary" + лист "totals"
        with pd.ExcelWriter(out_xlsx, engine="openpyxl") as writer:
            df_rep.to_excel(writer, sheet_name="summary", index=False)

            totals = {
                "files_processed": [len(df_rep)],
                "sum_raw_footprints": [int(df_rep["n_footprints_raw"].sum()) if "n_footprints_raw" in df_rep.columns else 0],
                "sum_buildings": [int(df_rep["n_buildings"].sum()) if "n_buildings" in df_rep.columns else 0],
                "sum_todelete": [int(df_rep["n_todelete"].sum()) if "n_todelete" in df_rep.columns else 0],
                "sum_detections": [int(df_rep["n_detections"].sum()) if "n_detections" in df_rep.columns else 0],
                "sum_matches": [int(df_rep["n_matches"].sum()) if "n_matches" in df_rep.columns else 0],
            }
            pd.DataFrame(totals).to_excel(writer, sheet_name="totals", index=False)

        print("Сохранила общий XLSX-репорт:", out_xlsx)

    print("\nВСЁ ГОТОВО ✅")
    print("Архивы лежат здесь:", OUT_ROOT)
    for r in results:
        print(f"- {r['base']}: {r['main_zip']}  |  {r['extras_zip']}")
    return results


# =========================================================
# 14) СТАРТ
# =========================================================
run_all()

Веса уже скачаны: /content/models/yolov8m-seg_LasVegas.pt bytes: 54828939
✅ Добавила safe globals: 34
⚠️ Safe-globals не сработало -> fallback weights_only=False. UnpicklingError('Weights only load failed. This file can still be loaded, to do so you have two options, \x1bdo those steps only if you trust the source of the checkpoint\x1b. \n\t(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.\n\t(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.\n\tWeightsUnpickler error: Unsupported global: GLOBAL ultralytics.nn.modules.block.C2f was not an allowed global by default. Please use `torch.serialization.add_safe_globals([ultralytics.nn.modules.block.C2f])` or the `torch.serializa

ms_buildings_1.geojsonl.gz: 100%|██████████| 10.1M/10.1M [00:00<00:00, 17.9MB/s]


[1/3] Парсинг GeoJSONL ...
    -> оставлено объектов: 215

[2/3] Скачивание...


ms_buildings_2.geojsonl.gz: 100%|██████████| 425k/425k [00:00<00:00, 2.35MB/s]


[2/3] Парсинг GeoJSONL ...
    -> оставлено объектов: 0

[3/3] Скачивание...


...

[1/2] Парсинг GeoJSONL ...
    -> оставлено объектов: 720

[2/2] Скачивание...


ms_buildings_2.geojsonl.gz: 100%|██████████| 27.4k/27.4k [00:00<00:00, 831kB/s]

[2/2] Парсинг GeoJSONL ...


    -> оставлено объектов: 0
Всего футпринтов (WGS84): 720
Всего футпринтов после фильтра по площади: 720
Сохранила GeoJSON: /content/out/tyrol-e36/tyrol-e36_raw.geojson (features=720)
Запускаю YOLO segmentation (тайлы)...
Детекций после склейки+NMS: 456
Футпринтов для матчинга: 656
Матчей: 412
Футпринтов без пары (ToDelete): 244
Детекций без пары (игнорируем): 44
Уточняю сдвиги по IoU против YOLO масок...
Сохранила отчёт по сдвигам: /content/out/tyrol-e36/tyrol-e36_shifts_report.csv
Сохранила GeoJSON: /content/out/tyrol-e36/tyrol-e36_Buildings_shifted.geojson (features=476)
Сохранила GeoJSON: /content/out/tyrol-e36/tyrol-e36_ToDelete.geojson (features=244)
Сохранила CVAT XML: /content/out/tyrol-e36/tyrol-e36_cvat_2labels.xml | Buildings: 476 | ToDelete: 244 | всего: 720
Сохранила PNG-отчёт: /content/out/tyrol-e36/tyrol-e36_report_buildings_todelete.png
Сохранила zip: /content/out/tyrol-e36.zip
Сохранила zip: /content/out/tyrol-e36_extras.zip
Удалена временная папка: /content/out/tyrol

[{'base': 'tyrol-e21',
  'work_dir': '/content/out/tyrol-e21',
  'main_zip': '/content/out/tyrol-e21.zip',
  'extras_zip': '/content/out/tyrol-e21_extras.zip',
  'grid_png': '/content/out/tyrol-e21/tyrol-e21_grid.png',
  'cvat_xml': '/content/out/tyrol-e21/tyrol-e21_cvat_2labels.xml',
  'report_png': '/content/out/tyrol-e21/tyrol-e21_report_buildings_todelete.png',
  'n_footprints_raw': 215,
  'n_buildings': 121,
  'n_todelete': 94,
  'n_detections': 123,
  'n_matches': 110,
  'n_unmatched_fp': 94,
  'n_unmatched_det': 13},
 {'base': 'tyrol-e22',
  'work_dir': '/content/out/tyrol-e22',
  'main_zip': '/content/out/tyrol-e22.zip',
  'extras_zip': '/content/out/tyrol-e22_extras.zip',
  'grid_png': '/content/out/tyrol-e22/tyrol-e22_grid.png',
  'cvat_xml': '/content/out/tyrol-e22/tyrol-e22_cvat_2labels.xml',
  'report_png': '/content/out/tyrol-e22/tyrol-e22_report_buildings_todelete.png',
  'n_footprints_raw': 716,
  'n_buildings': 422,
  'n_todelete': 294,
  'n_detections': 430,
  'n_matc

In [22]:
import shutil, os, glob

# 1) Очистить папку out
shutil.rmtree(OUT_ROOT, ignore_errors=True)
os.makedirs(OUT_ROOT, exist_ok=True)
print("✅ Папка OUT_ROOT очищена:", OUT_ROOT)

# 2) Очистить папку tifs
shutil.rmtree(INPUT_DIR, ignore_errors=True)
os.makedirs(INPUT_DIR, exist_ok=True)
print("✅ Папка INPUT_DIR очищена:", INPUT_DIR)

✅ Папка OUT_ROOT очищена: /content/out
✅ Папка INPUT_DIR очищена: /content/tifs
